In [16]:
from mt import *

In [17]:
test()

Hello world.


In [18]:
# The directory containing the images
data_dir = '../data/200129_double_hit_spinowitz/mz13-1'

# glob string for images
im_glob = os.path.join(data_dir, '*')

# Get list of images
im_list = sorted(glob.glob(im_glob))

im_list

['../data/200129_double_hit_spinowitz/mz13-1/200129_double_hit_spinowitz_mh13_1__w1-405 Confocal-Grace_s3.TIF',
 '../data/200129_double_hit_spinowitz/mz13-1/200129_double_hit_spinowitz_mh13_1__w1-405 Confocal-Grace_s4.TIF',
 '../data/200129_double_hit_spinowitz/mz13-1/200129_double_hit_spinowitz_mh13_1__w2-445 Confocal-Amjad_s3.TIF',
 '../data/200129_double_hit_spinowitz/mz13-1/200129_double_hit_spinowitz_mh13_1__w2-445 Confocal-Amjad_s4.TIF',
 '../data/200129_double_hit_spinowitz/mz13-1/200129_double_hit_spinowitz_mh13_1__w3-561 Confocal-Amjad_s3.TIF',
 '../data/200129_double_hit_spinowitz/mz13-1/200129_double_hit_spinowitz_mh13_1__w3-561 Confocal-Amjad_s4.TIF']

In [19]:
dapi_raw = skimage.io.imread(im_list[0])
cfp_raw = skimage.io.imread(im_list[2])
chfp_raw = skimage.io.imread(im_list[4])

In [20]:
dapi, cfp, chfp = best_slice(dapi_raw, cfp_raw, chfp_raw)

13


In [21]:
chfp = register_images(cfp, chfp)

Detected pixel offset (y, x): [ 1. -4.]


In [22]:
dapi_filt_gauss, cfp_filt_gauss, chfp_filt_gauss = filt_gauss(dapi, cfp, chfp, 1.5)

In [23]:
dapi_filt_gauss_bw, cfp_filt_gauss_bw, chfp_filt_gauss_bw, threshold2, threshold3 = threshold(dapi_filt_gauss, cfp_filt_gauss, chfp_filt_gauss)

0.03989297851185917 is where the dapi cutoff point is.
0.0101047412342507 is where the cfp cutoff point is.
0.042450265793501626 is where the chfp cutoff point is.


In [24]:
dapi_dil, cfp_dil, chfp_dil = dilate(dapi_filt_gauss_bw, cfp_filt_gauss_bw, chfp_filt_gauss_bw, 0)

In [25]:
label_image_dapi, df_dapi, dapi_area = ws(dapi_dil, 50, dapi)
label_image_cfp, df_cfp, cfp_area = ws(cfp_dil, 50, cfp)
label_image_chfp, df_chfp, chfp_area = ws(chfp_dil, 50, chfp)

In [26]:
cfp_large_sub, df_cfp_large = remove_large(label_image_cfp, 1000, df_cfp, cfp)
chfp_large_sub, df_chfp_large = remove_large(label_image_chfp, 1000, df_chfp, chfp)

In [27]:
print('Percent CFP+ area:', cfp_area/dapi_area)
print('Percent ChFP+ area:', chfp_area/dapi_area)

Percent CFP+ area: 0.135745579144661
Percent ChFP+ area: 0.055943174013092796


In [28]:
df_co = coloc(cfp_large_sub, chfp_large_sub, cfp, chfp)

In [29]:
all_plot = hv.Points(
    data=df_co,
    kdims=['CFP mean intensity', 'ChFP mean intensity'],
    vdims=[],
    label='All nuclei'
).opts()
all_plot

:Points   [CFP mean intensity,ChFP mean intensity]

In [30]:
x = df_co['CFP mean intensity'].values.reshape((-1, 1))
y = df_co['ChFP mean intensity'].values
model = linear_model.LinearRegression().fit(x, y)
r_sq = model.score(x, y)
print('R squared value is:', r_sq)

R squared value is: 0.18606581797148813


In [32]:
cfp_thresh = threshold2 * 65535
chfp_thresh = threshold3 * 65535
print(cfp_thresh, chfp_thresh)

662.2142167866197 2781.978168777129


In [33]:
df_cfp_chfp = df_co.loc[(df_co.loc[:, 'CFP mean intensity'] > cfp_thresh) &
                          (df_co.loc[:, 'ChFP mean intensity'] > chfp_thresh)]
df_cfp = df_co.loc[(df_co.loc[:, 'CFP mean intensity'] > cfp_thresh) &
                          (df_co.loc[:, 'ChFP mean intensity'] < chfp_thresh)]
df_chfp = df_co.loc[(df_co.loc[:, 'CFP mean intensity'] < cfp_thresh) &
                          (df_co.loc[:, 'ChFP mean intensity'] > chfp_thresh)]

all_plot = hv.Points(
    data=df_co,
    kdims=['CFP mean intensity', 'ChFP mean intensity'],
    vdims=[],
    label='All nuclei'
).opts(color='black', alpha=0.6)

cfp_plot = hv.Points(
    data=df_cfp,
    kdims=['CFP mean intensity', 'ChFP mean intensity'],
    vdims=[],
    label='CFP+ only nuclei'
).opts(color=colorcet.glasbey_dark[3], alpha=0.6)

chfp_plot = hv.Points(
    data=df_chfp,
    kdims=['CFP mean intensity', 'ChFP mean intensity'],
    vdims=[],
    label='mCh+ only Nuclei'
).opts(color=colorcet.glasbey_dark[13], alpha=0.6)

cfp_chfp_plot = hv.Points(
    data=df_cfp_chfp,
    kdims=['CFP mean intensity', 'ChFP mean intensity'],
    vdims=[],
    label='CFP+/mCh+ nuclei'
).opts(color=colorcet.glasbey_dark[4], alpha=0.8)

chfp_plot*cfp_plot*cfp_chfp_plot

:Overlay
   .Points.MCh_plus_only_Nuclei          :Points   [CFP mean intensity,ChFP mean intensity]
   .Points.CFP_plus_only_nuclei          :Points   [CFP mean intensity,ChFP mean intensity]
   .Points.CFP_plus_over_mCh_plus_nuclei :Points   [CFP mean intensity,ChFP mean intensity]

In [34]:
cfp_line = np.linspace(1000, 9000, 200)
chfp_line = np.linspace(500, 1400, 200)

cfp = (cfp_thresh, cfp_line)
chfp = (chfp_line, chfp_thresh)

c = hv.Path(cfp).opts(line_color='black', 
                          line_dash='dotted', 
                          line_alpha=0.5)
ch = hv.Path(chfp).opts(line_color='black', 
                          line_dash='dotted', 
                          line_alpha=0.5)

chfp_plot*cfp_plot*cfp_chfp_plot*c*ch

:Overlay
   .Points.MCh_plus_only_Nuclei          :Points   [CFP mean intensity,ChFP mean intensity]
   .Points.CFP_plus_only_nuclei          :Points   [CFP mean intensity,ChFP mean intensity]
   .Points.CFP_plus_over_mCh_plus_nuclei :Points   [CFP mean intensity,ChFP mean intensity]
   .Path.I                               :Path   [x,y]
   .Path.II                              :Path   [x,y]

In [35]:
total_cells = len(df_co.index)
co_cells = len(df_cfp_chfp.index)
cfp_only_cells = len(df_cfp.index)
chfp_only_cells = len(df_chfp.index)

print('Total nuclei:', total_cells)
print('Number of CFP+/mCh+ cells:', co_cells)
print('Number of CFP+ only cells:', cfp_only_cells)
print('Number of mCh+ only cells:', chfp_only_cells)

Total nuclei: 147
Number of CFP+/mCh+ cells: 19
Number of CFP+ only cells: 90
Number of mCh+ only cells: 38


In [36]:
print('% of CFP+/mCh+ cells:', co_cells / total_cells * 100)
print('% of CFP+ only cells:', cfp_only_cells / total_cells * 100)
print('% of mCh+ only cells:', chfp_only_cells / total_cells * 100)

% of CFP+/mCh+ cells: 12.925170068027212
% of CFP+ only cells: 61.224489795918366
% of mCh+ only cells: 25.850340136054424
